### 利用Bert微调Question Answering任务
1. 准备数据
* 对于QA任务，数据通常包括问题（question）、上下文（context）和答案（answer）。上下文中包含问题的答案，而答案在上下文中的位置需要通过字符偏移来标记。以SQuAD（Stanford Question Answering Dataset）为例，数据集包含了多个问题-上下文-答案的样本。
2. 选择预训练模型和分词器
* 我们使用Hugging Face的Transformers库加载BERT模型及其分词器（如bert-base-uncased）。分词器负责将问题和上下文拼接并转为模型输入的token。
3. 数据预处理
* 在QA任务中，需要将问题和上下文分词，并处理以下几方面：
    * Tokenize问题和上下文：将问题和上下文拼接成模型输入，并进行分词。分词后确保长度不超过模型的最大输入限制。
    * 标记答案位置：将答案的字符位置转换为token的位置索引，标记出答案的开始和结束位置。
    * 无答案情况：在一些数据集中（如SQuAD v2），问题可能没有答案，这时设置答案的开始和结束位置为0，以便模型能够学习识别无答案的情况。
4. 设置训练参数
5. 微调模型
* 使用Trainer API来管理微调过程。Trainer接受模型、训练集和验证集，并使用指定的超参数来执行训练和评估。这一API封装了常见的训练步骤，比如前向传播、反向传播、梯度更新等，大大简化了微调流程。
6. 验证与评估模型
* 微调完成后，可以在验证集上评估模型表现。通过设定验证指标（如F1-score或准确率），可以检查模型是否有效地找到了答案。
7. 进行推理
* 微调后的模型可以用于实际的QA任务，即输入新的问题和上下文，模型可以根据上下文找到并输出答案。这通常通过输入问题和上下文，模型生成预测的开始和结束位置，将对应的token转换为答案文本。

### Bert是怎么实现Question Answering任务的？
BERT实现Question Answering（QA）任务时，通过输入问题（Question）和上下文（Context）来预测答案在上下文中的位置。BERT在QA任务中的输出为答案在上下文中的“起始位置”和“结束位置”的概率，这些概率指向了上下文中包含答案的子句或片段
1. 输入格式
* 在QA任务中，输入数据由一个问题和一个上下文组成，将二者进行拼接，构成BERT的标准输入格式：[CLS] 问题 [SEP] 上下文 [SEP]
    * [CLS]是一个特殊的起始标志，BERT在预训练时会对这个标志的输出进行分类任务，因此可以在QA任务中使用它来指引模型。
    * [SEP]是分隔符，用来区分问题和上下文。
2. Bert的输出
* 输入经过编码后会生成对应的输出向量。假设输入有n个token，BERT会生成一个n个token对应的输出向量。对于QA任务，每个token的位置向量经过进一步的处理，用于预测它是否为答案的起始或结束位置。
3. 答案起始和结束位置预测
* BERT在QA任务中增加了两个分类头（Classification Head），用于预测答案的“起始位置”和“结束位置”。这些头部是两个线性层，分别计算每个token是答案起始或结束位置的概率。
    * 起始位置预测：BERT通过一个线性层计算每个token是答案起始位置的概率。输出的概率分布表示每个token作为答案起始的可能性。
    * 结束位置预测：同样，BERT使用另一个线性层计算每个token是答案结束位置的概率。
* 模型在训练时会学习到每个token的输出向量与答案位置之间的对应关系，并通过计算交叉熵损失函数，使预测结果与实际答案位置接近。
4. 损失函数
* QA任务中的损失函数由起始位置和结束位置的预测损失共同组成，具体计算方式为：
    * Loss=CrossEntropyLoss(start_logits,true_start)+CrossEntropyLoss(end_logits,true_end)
        * start_logits和end_logits分别是模型预测的每个token作为答案起始或结束位置的概率分布。
        * true_start和true_end分别是标签数据中的真实答案起始和结束位置。


### 1. 导包

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from datasets import load_dataset

/Users/bytedance/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/bytedance/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. 导入数据集

In [2]:
# 加载数据集，这里以SQuAD v2为例
dataset = load_dataset("squad_v2")

Generating validation split: 100%|██████████| 11873/11873 [00:00<00:00, 1025577.60 examples/s]


### 3. 加载模型与分词器

In [3]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 4. 数据预处理

In [4]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        padding="max_length",
        return_offsets_mapping=True,
        return_tensors="pt"
    )
    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]

    start_positions = []
    end_positions = []
    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        if len(answer["answer_start"]) == 0:
            start_positions.append(0)
            end_positions.append(0)
        else:
            start_char = answer["answer_start"][0]
            end_char = start_char + len(answer["text"][0])
            sequence_ids = inputs.sequence_ids(i)

            # 将字符位置映射到token位置
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1
            token_end_index = len(sequence_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if not (offset[token_start_index][0] <= start_char and offset[token_end_index][1] >= end_char):
                start_positions.append(0)
                end_positions.append(0)
            else:
                while token_start_index < len(offset) and offset[token_start_index][0] <= start_char:
                    token_start_index += 1
                start_positions.append(token_start_index - 1)

                while offset[token_end_index][1] >= end_char:
                    token_end_index -= 1
                end_positions.append(token_end_index + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)


Map: 100%|██████████| 11873/11873 [00:09<00:00, 1295.34 examples/s]


### 5. 设置训练参数

In [5]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

/Users/bytedance/Library/Python/3.9/lib/python/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### 6. 训练模型

In [6]:
# 没GPU，跑的太慢了，先不跑了
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)
trainer.train()

  2%|▏         | 500/24435 [15:57<10:27:46,  1.57s/it] 

{'loss': 2.6578, 'grad_norm': 12.769356727600098, 'learning_rate': 1.9590750971966443e-05, 'epoch': 0.06}


  4%|▍         | 1000/24435 [28:47<9:56:02,  1.53s/it]

{'loss': 1.7902, 'grad_norm': 22.286327362060547, 'learning_rate': 1.9181501943932885e-05, 'epoch': 0.12}


  6%|▌         | 1483/24435 [41:00<9:59:37,  1.57s/it] 

KeyboardInterrupt: 

### 7. 验证和测试模型

In [ ]:
results = trainer.evaluate()
print(results)

### 8. 使用微调后的模型进行推理

In [ ]:
def answer_question(question, context):
    inputs = tokenizer(question, context, return_tensors="pt")
    outputs = model(**inputs)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
    return answer

# 示例
question = "What is Transformers library?"
context = "The Transformers library by Hugging Face is a collection of pretrained models for NLP."
print(answer_question(question, context))
